In [ ]:

import os
import time
import pandas as pd
import hashlib
import magic
import re
        

def file2hash(filename, hash_function):
    with open(filename, 'rb', buffering=0) as f:
        return hash_function(f.read()).hexdigest()


# specify the directory path where the files are located
dir_path = 'C:'

# create an empty list to store the file names
file_names = []
extensions = []
md5s = []
sha1s = []
sha256s = []
magic_numbers = []
magic_object = magic.Magic(mime=True)
usb_devices = []
creation_times = []
modification_times = []
access_times = []

with open("C:\\Users\\A507\\RF\\setupapi.dev2.log", "r") as log_file:
     # Store information about each USB device in a dictionary
     for line in log_file:
     # Find all USB device installation events and extract information about each device
        match = re.match(r'^>>>  \[Device Install.*#(Disk&Ven_[A-Za-z0-9]+)&(Prod_([\w\s\S]+?))&(Rev_([\w\s\S]+?))#([\w\s\S]+?)#.*\]', line)
        if(not match):
            continue
        device_vendor_id = match.group(1)
        device_product_id = match.group(2)
        device_instance_id = match.group(4)
        serial_number = match.group(6)
        event_time = next(log_file).split("start")[1].split("\n")[0]
        usb_devices.append({"device_vendor_id" : device_vendor_id, "device_product_id" : device_product_id, "device_instance_id" : device_instance_id, "serial_number" : serial_number, "event_time" : event_time})
# iterate through all files in the directory
for file in os.listdir(dir_path):
    # check if the file is a regular file (i.e., not a directory)
    path = os.path.join(dir_path, file)
    if os.path.isfile(os.path.join(dir_path, file)):
        # if so, add the file name to the list
        file_names.append(os.path.splitext(file)[0])
        extensions.append(os.path.splitext(file)[1])
        md5s.append(file2hash(path, hashlib.md5))
        sha1s.append(file2hash(path, hashlib.sha1))
        sha256s.append(file2hash(path, hashlib.sha256))
        magic_number = magic_object.from_file(path)
        magic_numbers.append(magic_number)
        extension = os.path.splitext(file)[1]
        creation_times = time.ctime(os.path.getctime(os.path.join(dir_path, file)))
        modification_times = time.ctime(os.path.getmtime(os.path.join(dir_path, file)))
        access_times = time.ctime(os.path.getatime(os.path.join(dir_path, file)))
        
# create a Pandas dataframe with the file names
dfusb = pd.DataFrame({'file_name' : file_names, 'extension' : extensions, 'md5' : md5s, 'sha1' : sha1s, 'sha256' : sha256s, 'magic_number' : magic_numbers, 'creation_time' : creation_times, 'modification_time' : modification_times, 'access_time' : access_times})
df = pd.DataFrame(usb_devices)
display(dfusb)
display(df)